In [2]:
import bigframes
bigframes.options.experiments.blob = True
#bigframes.options._bigquery_options.client_endpoints_override = {"bqclient": "https://test-bigquery.sandbox.google.com", 
#                                                           "bqconnectionclient": "test-bigqueryconnection.sandbox.googleapis.com", 
#                                                           "bqstoragereadclient": "test-bigquerystorage-grpc.sandbox.googleapis.com"}
import bigframes.pandas as bpd

/usr/local/google/home/shuowei/src/python-bigquery-dataframes/bigframes/_config/experiment_options.py:69: PreviewWarning: BigFrames Blob is still under experiments. It may not work and subject
to change in the future.
  warnings.warn(msg, category=bfe.PreviewWarning)


In [3]:
extract_df = bpd.from_glob_path("gs://bigframes_blob_test/pdfs/*", name="pdf")
extract_df = extract_df.head(5)

/usr/local/google/home/shuowei/src/python-bigquery-dataframes/bigframes/core/global_session.py:114: DefaultLocationWarning: No explicit location is set, so using location US for the session.
  return func(get_global_session(), *args, **kwargs)


In [4]:
bq_connection = "bigframes-dev.us.bigframes-default-connection"
extract_df["extract_text"] = extract_df["pdf"].blob.pdf_extract(
    connection=bq_connection, verbose=True)

In [4]:
extract_df

,pdf,extract_text
0,"uri: gs://bigframes_blob_test/pdfs/test-protected.pdf, authorizer: bigframes-dev.us.bigframes-default-connection","{'status': 'File has not been decrypted', 'content': ''}"
1,"uri: gs://bigframes_blob_test/pdfs/sample-local-pdf.pdf, authorizer: bigframes-dev.us.bigframes-default-connection","{'status': '', 'content': 'Sample PDF This is a testing file. Some dummy messages are used for testing purposes. '}"


In [5]:
extract_df["extract_text"].explode()

0    {'status': 'File has not been decrypted', 'con...
1    {'status': '', 'content': 'Sample  PDF    This...
Name: extract_text, dtype: struct<status: string, content: string>[pyarrow]

In [6]:
extract_df["extract_text"]

0    {'status': 'File has not been decrypted', 'con...
1    {'status': '', 'content': 'Sample  PDF    This...
Name: extract_text, dtype: struct<status: string, content: string>[pyarrow]

In [16]:
extract_df["pdf"].blob.pdf_chunk(connection=bq_connection, verbose=False).explode().to_pandas()

/usr/local/google/home/shuowei/src/python-bigquery-dataframes/bigframes/operations/blob.py:736: FunctionAxisOnePreviewWarning: Blob Functions use bigframes DataFrame Managed function with axis=1 senario, which is a preview feature.
  


0                                                 <NA>
1    Sample  PDF    This  is  a  testing  file.  So...
dtype: string

In [6]:
extract_df["test"]

0                                                     
1    Sample  PDF    This  is  a  testing  file.  So...
Name: test, dtype: string

In [20]:
import pandas as pd

chunk_data = [
    {"status": "File has not been decrypted", "content": []},
    {
        "status": "",
        "content": ["Sample  PDF   This  is  a  testing  file.",  "Some  dummy  messages  are  used  for  testing  purposes.  "],
    },
]


content_values = []
for item in chunk_data:
    if not item["content"]:
        content_values.append(pd.NA)
    else:
        content_values.extend(item["content"])

expected = pd.Series(content_values)
print(expected)


0                                                 <NA>
1            Sample  PDF   This  is  a  testing  file.
2    Some  dummy  messages  are  used  for  testing...
dtype: object
